<a href="https://colab.research.google.com/github/amit-deokar/mist6150/blob/master/Ch14_Example_US_Baby_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Textbook Chapter 14: Data Analysis Examples

**NOTE**: Use the following cell as standard import. You may need to ocassionally import additional Python packages, but this should work for most parts.

In [0]:
!pip install -U pandas

Import standard cells:

In [0]:
# Some standard imports

import os

# scipy imports
# There are several universal functions for numpy arrays that are available through the scipy package
import scipy as sc
from scipy import stats, integrate
from scipy.stats.mstats import mode

# numpy imports
# pandas depends on numpy
import numpy as np
np.set_printoptions(precision=4, threshold=500, suppress=True)
np.random.seed(12345)
np.random.seed(sum(map(ord, "distributions")))

# pandas imports
# The convention is to import pandas package with a pd prefix. 
# Also, since we most commonly use Series and DataFrame classes from this package, 
# we import them into the current namespace, so we do not have to refer to them with the pd prefix.
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('display.max_columns', None) # enables showing all columns
pd.options.display.max_rows = 20
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.notebook_repr_html = True

# matplotlib imports
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
plt.subplots(figsize=(10,6))
%matplotlib inline

# seaborn imports
import seaborn as sns
sns.set(color_codes=True)

# bokeh imports
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once') #enable if needed to see the warning the first time.

You might see the following warning:
```
FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
```
That warning has something to do with the `Seaborn` libary needing to adjust their code to be compatible to the latest version of `Pandas` API. It will likely be fixed in near future. You can ignore that warning right now.

In [0]:
pd.__version__

## Example 3. US Baby Names 1880–2010

The United States Social Security Administration (SSA) has made available data on the frequency of baby names from 1880 through the present. Hadley Wickham, an author of several popular R packages, has often made use of this dataset in illustrating data manipulation in R.

We need to do some data wrangling to load this dataset, but once we do that we will have a DataFrame that looks like this:

```
In [4]: names.head(10)
Out[4]:
        name sex  births  year
0       Mary   F    7065  1880
1       Anna   F    2604  1880
2       Emma   F    2003  1880
3  Elizabeth   F    1939  1880
4     Minnie   F    1746  1880
5   Margaret   F    1578  1880
6        Ida   F    1472  1880
7      Alice   F    1414  1880
8     Bertha   F    1320  1880
9      Sarah   F    1288  1880
```

There are many things you might want to do with the dataset:
- Visualize the proportion of babies given a particular name (your own, or another name) over time
- Determine the relative rank of a name
- Determine the most popular names in each year or the names whose popularity has advanced or declined the most
- Analyze trends in names: vowels, consonants, length, overall diversity, changes in spelling, first and last letters
- Analyze external sources of trends: biblical names, celebrities, demographic changes

With the tools in this book, many of these kinds of analyses are within reach, so I will walk you through some of them.
As of this writing, the US Social Security Administration makes available data files, one per year, containing the total number of births for each sex/name combination. The raw archive of these files can be obtained from http://www.ssa.gov/oact/babynames/limits.html.

In the event that this page has been moved by the time you’re reading this, it can most likely be located again by an internet search. After downloading the "National data" file `names.zip` and unzipping it, you will have a directory containing a series of files like `yob1880.txt`. 

We can use the Unix head command to look at the first 10 lines of one of the files (on Windows, you can use the more command or open it in a text editor). Alternatively, we can use magic command from within the Jupyter notebook.

In [0]:
!head -n 10 ../data/babynames/yob1880.txt

As this is already in a nicely comma-separated form, it can be loaded into a DataFrame with `pandas.read_csv`:

In [0]:
import os

currdir = %pwd
datadir = os.path.join(currdir, '../data/babynames/')
datadir

In [0]:
import pandas as pd
names1880 = pd.read_csv(datadir + 'yob1880.txt', names=['name', 'sex', 'births'])
names1880

These files only contain names with at least five occurrences in each year, so for simplicity’s sake we can use the sum of the births column by sex as the total number of births in that year:

In [0]:
names1880.groupby('sex').births.sum()

Since the dataset is split into files by year, one of the first things to do is to assemble all of the data into a single DataFrame and further to add a year field. You can do this using `pandas.concat`:

In [0]:
years = range(1880, 2011)

pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = datadir + 'yob%d.txt' % year
    frame = pd.read_csv(path, names=columns)

    frame['year'] = year
    pieces.append(frame)

There are a couple things to note here. First, remember that concat glues the DataFrame objects together row-wise by default. Secondly, you have to pass `ignore_index=True` because we’re not interested in preserving the original row numbers returned from `read_csv`. So we now have a very large DataFrame containing all of the names data:

In [0]:
# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)
names

### Dask for parallel computing in Python

[Dask](https://docs.dask.org/) is a flexible library for parallel computing in Python.

Dask is composed of two parts:

- Dynamic task scheduling optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
- “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.

Dask emphasizes the following virtues:

- Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
- Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
- Native: Enables distributed computing in pure Python with access to the PyData stack.
- Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
- Scales up: Runs resiliently on clusters with 1000s of cores
- Scales down: Trivial to set up and run on a laptop in a single process
- Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans

#### [Common Uses and Anti-Uses](https://docs.dask.org/en/latest/dataframe.html)

Dask DataFrame is used in situations where Pandas is commonly needed, usually when Pandas fails due to data size or computation speed:

- Manipulating large datasets, even when those datasets don’t fit in memory
- Accelerating long computations by using many cores
- Distributed computing on large datasets with standard Pandas operations like groupby, join, and time series computations

Dask DataFrame may not be the best choice in the following situations:

- If your dataset fits comfortably into RAM on your laptop, then you may be better off just using Pandas. There may be simpler ways to improve performance than through parallelism
- If your dataset doesn’t fit neatly into the Pandas tabular model, then you might find more use in dask.bag or dask.array
- If you need functions that are not implemented in Dask DataFrame, then you might want to look at dask.delayed which offers more flexibility
- If you need a proper database with all that databases offer you might prefer something like Postgres

In [0]:
!pip install "dask[complete]"

In [0]:
import dask.dataframe as dd

years = range(1880, 2011)

pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = datadir + 'yob%d.txt' % year
    frame = dd.read_csv(path, names=columns)

    frame['year'] = year
    pieces.append(frame)

In [0]:
pieces[:3]

NOTE: Dask provides a [concatenate](http://docs.dask.org/en/latest/array-stack.html) function as well. It automatically ignores the indexes of objects to be concatenated.

In [0]:
# Concatenate everything into a single Dask DataFrame 
names = dd.concat(pieces)
names

In [0]:
type(names)

In [0]:
names.head()

In [0]:
names = names.categorize("sex") # for dask dataframes, columns must be categorical, for pandas dataframes it is automatically inferred

total_births = names.pivot_table(values='births', index='year', columns='sex', aggfunc='sum').compute() # Note: We are now converting to a pandas DataFrame using the compute() method.
total_births.tail()

In [0]:
total_births.plot(title='Total births by sex and year')

### Reverting back to pandas DataFrame

It is easy to start using pandas API at any point from a Dask DataFrame.

In [0]:
names = names.compute()
type(names)

In [0]:
names.groupby(['year', 'sex']).sum()

Next, let’s insert a column `prop` with the fraction of babies given each name relative to the total number of births. A prop value of `0.02` would indicate that 2 out of every 100 babies were given a particular name. Thus, we group the data by year and sex, then add the new column to each group:

In [0]:
def add_prop(group):
    group['prop'] = group.births / group.births.sum()
    return group

In [0]:
names = names.groupby(['year', 'sex']).apply(add_prop)
names

When performing a group operation like this, it’s often valuable to do a sanity check, like verifying that the `prop` column sums to `1` within all the groups:

In [0]:
names.groupby(['year', 'sex']).prop.sum()

Now that this is done, I’m going to extract a subset of the data to facilitate further analysis: the top 1,000 names for each sex/year combination. This is yet another group operation:

In [0]:
def get_top1000(group):
    return group.sort_values(by='births', ascending=False)[:1000]

grouped = names.groupby(['year', 'sex'])

top1000 = grouped.apply(get_top1000)

# Drop the group index, not needed
top1000.reset_index(inplace=True, drop=True)

If you prefer a do-it-yourself approach, try this instead:

In [0]:
pieces = []
for year, group in names.groupby(['year', 'sex']):
    pieces.append(group.sort_values(by='births', ascending=False)[:1000])
top1000 = pd.concat(pieces, ignore_index=True)

The resulting dataset is now quite a bit smaller:

In [0]:
top1000

### Analyzing Naming Trends

With the full dataset and Top 1,000 dataset in hand, we can start analyzing various naming trends of interest. Splitting the Top 1,000 names into the boy and girl portions is easy to do first:

In [0]:
boys = top1000[top1000.sex == 'M']
girls = top1000[top1000.sex == 'F']

Simple time series, like the number of Johns or Marys for each year, can be plotted but require a bit of munging to be more useful. Let’s form a pivot table of the total number of births by year and name:

In [0]:
total_births = top1000.pivot_table(values='births', index='year', columns='name', aggfunc=sum)

In [0]:
total_births.info()

In [0]:
total_births.head()

In [0]:
subset = total_births[['Tom', 'Dick', 'Harry', 'John', 'Mary', 'Marilyn']]

subset.plot(subplots=True, figsize=(20, 20), grid=False, title="Number of births per year")

On looking at this, you might conclude that these names have grown out of favor with the American population. But the story is actually more complicated than that, as will be explored in the next section.

#### Measuring the increase in naming diversity

One explanation for the decrease in plots is that fewer parents are choosing common names for their children. This hypothesis can be explored and confirmed in the data. One measure is the proportion of births represented by the top 1,000 most popular names, which I aggregate and plot by year and sex:

In [0]:
plt.figure()

In [0]:
table = top1000.pivot_table(values='prop', index='year', columns='sex', aggfunc=sum)

table.plot(title='Sum of table1000.prop by year and sex',
           yticks=np.linspace(0, 1.2, 13), xticks=range(1880, 2020, 10),
           figsize=(14, 7), grid = True)

You can see that, indeed, there appears to be increasing name diversity (decreasing total proportion in the top 1,000). Another interesting metric is the number of distinct names, taken in order of popularity from highest to lowest, in the top 50% of births. This number is a bit more tricky to compute. Let’s consider just the boy names from 2010:

In [0]:
df = boys[boys.year == 2010]
df

After sorting prop in descending order, we want to know how many of the most popular names it takes to reach 50%. You could write a `for` loop to do this, but a vectorized NumPy way is a bit more clever. Taking the cumulative sum, `cumsum`, of prop and then calling the method `searchsorted` returns the position in the cumulative sum at which 0.5 would need to be inserted to keep it in sorted order:

In [0]:
prop_cumsum = df.sort_values(by='prop', ascending=False).prop.cumsum()

prop_cumsum[:10]

Since arrays are zero-indexed, adding 1 to this result gives you a result of 117. By contrast, in 1900 this number was much smaller as shown later:

In [0]:
prop_cumsum.values.searchsorted(0.5) + 1

In [0]:
df = boys[boys.year == 1900]

in1900 = df.sort_values(by='prop', ascending=False).prop.cumsum()

In [0]:
in1900.values.searchsorted(0.5) + 1

You can now apply this operation to each year/sex combination, groupby those fields, and apply a function returning the count for each group:

In [0]:
def get_quantile_count(group, q=0.5):
    group = group.sort_values(by='prop', ascending=False)
    return group.prop.cumsum().values.searchsorted(q) + 1

In [0]:
diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)
diversity

Upon unstacking, this resulting DataFrame diversity now has two time series, one for each sex, indexed by year. This can be inspected and plotted as before.

In [0]:
diversity = diversity.unstack('sex')
diversity

In [0]:
fig = plt.figure()
diversity.plot(title="Number of popular names in top 50%", figsize=(14, 7))

As you can see, girl names have always been more diverse than boy names, and they have only become more so over time. Further analysis of what exactly is driving the diversity, like the increase of alternative spellings, is left to the reader.

#### The “last letter” revolution

In 2007, baby name researcher Laura Wattenberg pointed out on her [website](http://www.babynamewizard.com/) that the distribution of boy names by final letter has changed significantly over the last 100 years. To see this, we first aggregate all of the births in the full dataset by year, sex, and final letter:

In [0]:
# extract last letter from name column
get_last_letter = lambda x: x[-1]

last_letters = names.name.map(get_last_letter)
last_letters.name = 'last_letter' # naming the series

last_letters

##### Sidenote: `apply` vs `applymap` vs `map`.

- `DataFrame.apply` operates on entire rows or columns at a time.
- `DataFrame.applymap`, `Series.apply`, and `Series.map` operate on one element at time.

There is a lot of overlap between the capabilities of `Series.apply` and `Series.map`, meaning that either one will work in most cases. They do have some slight differences though, some of which are discussed in this [post](https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas).

In [0]:
table = names.pivot_table(values='births', index=last_letters, columns=['sex', 'year'], aggfunc='sum')
table

Then we select out three representative years spanning the history and print the first few rows:

In [0]:
subtable = table.reindex(columns=[1910, 1960, 2010], level='year')
subtable.head()

Next, normalize the table by total births to compute a new table containing proportion of total births for each sex ending in each letter:

In [0]:
subtable.sum()

In [0]:
letter_prop = subtable / subtable.sum()
letter_prop

In [0]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(16, 16))
letter_prop['M'].plot(kind='bar', rot=0, ax=axes[0], title="Proportion of boy names ending in each letter")
letter_prop['F'].plot(kind='bar', rot=0, ax=axes[1], title="Proportion of girl names ending in each letter" , legend=False)

As you can see, boy names ending in `n` have experienced significant growth since the 1960s. Going back to the full table created before, we can again normalize by year and sex and select a subset of letters for the boy names, finally transposing to make each col‐ umn a time series:

In [0]:
plt.subplots_adjust(hspace=0.25)

In [0]:
letter_prop = table / table.sum()
dny_ts = letter_prop.loc[['d', 'n', 'y'], 'M'].T
dny_ts.head()

With this DataFrame of time series in hand, we can make a plot of the trends over time again with its plot method.

In [0]:
plt.close('all')
fig = plt.figure()

In [0]:
dny_ts.plot(figsize=(16, 10), title="Proportion of boys born with names ending in d/n/y over time")

#### Boy names that became girl names (and vice versa)

Another fun trend is looking at boy names that were more popular with one sex earlier in the sample but have “changed sexes” in the present. One example is the name Lesley or Leslie. Going back to the top1000 DataFrame, We can compute a list of names occurring in the dataset starting with "lesl":

In [0]:
all_names = pd.Series(top1000.name.unique())

In [0]:
lesley_like = all_names[all_names.str.lower().str.contains('lesl')]
lesley_like

From there, we can filter down to just those names and sum births grouped by name to see the relative frequencies:

In [0]:
filtered = top1000[top1000.name.isin(lesley_like)]

filtered.groupby('name').births.sum()

Next, let’s aggregate by sex and year and normalize within year:

In [0]:
table = filtered.pivot_table(values='births', index='year', columns='sex', aggfunc='sum')
table

In [0]:
table.sum(1)

In [0]:
table = table.div(table.sum(1), axis=0) # DataFrame.div() performs floating division of dataframe; DataFrame.sum(1) returns the sum of the values for axis 1, i.e., columns.
table

Lastly, it’s now possible to make a plot of the breakdown by sex over time.

In [0]:
fig = plt.figure()
table.plot(figsize=(16, 10), title = "Proportion of male/female Lesley-like names over time")